# 项目：整理Netflix电影演员评分数据

## 分析目标

此数据分析的目的是，整理不同流派影视作品，比如喜剧片、动作片、科幻片中，各演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。

本实战项目的目的在于练习整理数据，从而得到可供下一步分析的数据。

## 简介

原始数据集记录了截止至2022年7月美国地区可观看的所有Netflix电视剧及电影数据。数据集包含两个数据表：`titles.csv`和`credits.csv`。

`titles.csv`包含电影及电视剧相关信息，包括影视作品ID、标题、类型、描述、流派、IMDB（一个国外的在线评分网站）评分，等等。`credits.csv`包含超过7万名出现在Netflix影视作品的导演及演员信息，包括名字、影视作品ID、人物名、演职员类型（导演/演员）等。

`titles.csv`每列的含义如下：
- id：影视作品ID。
- title：影视作品标题。
- show_type：作品类型，电视节目或电影。
- description：简短描述。
- release_year：发布年份。
- age_certification：适龄认证。
- runtime：每集电视剧或电影的长度。
- genres：流派类型列表。
- production_countries：出品国家列表。
- seasons：如果是电视剧，则是季数。
- imdb_id：IMDB的ID。
- imdb_score：IMDB的评分。
- imdb_votes：IMDB的投票数。
- tmdb_popularity：TMDB的流行度。
- tmdb_score：TMDB的评分。

`credits.csv`每列的含义如下：
- person_ID：演职员ID。
- id：参与的影视作品ID。
- name：姓名。
- character_name：角色姓名。
- role：演职员类型，演员或导演。

## 读取数据

In [23]:
import pandas as pd

In [24]:
original_titles_data = pd.read_csv('titles.csv')
original_titles_data.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


In [25]:
original_credits_data = pd.read_csv('credits.csv')
original_credits_data.head()

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR


## 评估和清洗数据

评估数据分为两个方向：  
- 整齐度，即每列代表一个变量，每行代表一个观察值，每个单元格代表一个值；
- 干净度，即是否存在影响分析目标的缺失数据、重复数据、不一致数据和无效或失效数据。

`titles.csv`表中与分析目标有关的列有：`id` `title` `genres` `imdb_score`;  
`credits.csv`表中与分析目标有关的列有：`person_id` `id` `name` `role`

为了区分开经过清理的数据和原始的数据，我们创建新的变量`cleaned_titles`，让它为`original_titles`复制出的副本，以及创建新的变量`cleaned_credits`，让它为`original_credits`复制出的变量。我们之后的清理步骤都将被运用在`cleaned_titles`和`cleaned_credits`上。

In [26]:
cleaned_titles = original_titles_data.copy()
cleaned_credits = original_credits_data.copy()

### 整齐度评估和清洗

- 对`titles.csv`电影表进行评估，随机提取10行观察

In [27]:
cleaned_titles.sample(10)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1345,ts37896,Conspiracy,SHOW,Skeptics and others discuss widely held conspi...,2015,TV-14,43,"['history', 'documentation', 'european']",['GB'],1.0,tt4761054,6.2,379.0,2.421,5.8
5330,ts306068,Luna Park,SHOW,"In 1960s Rome, a meeting between a free-spirit...",2021,TV-MA,50,"['drama', 'romance']",['IT'],1.0,tt15349110,6.4,721.0,4.545,8.1
3866,ts296153,Ashley Garcia: Genius in Love,SHOW,15-year-old scientist Ashley Garcia explores t...,2020,TV-PG,29,['comedy'],['US'],3.0,tt10380934,6.7,1211.0,11.279,7.8
5652,tm990740,What Would Sophia Loren Do?,MOVIE,"Nancy Vincenza Kulik, an Italian-American gran...",2021,NaN,33,['documentation'],['US'],NaN,NaN,NaN,NaN,3.934,6.7
3828,ts267956,Ethos,SHOW,A group of unique characters from dramatically...,2020,NaN,49,"['drama', 'thriller']",['TR'],1.0,NaN,NaN,NaN,8.766,7.9
5751,tm1146635,Daiki Tsuneta: Tokyo Chaotic,MOVIE,This documentary follows King Gnu frontman Dai...,2021,NaN,89,"['documentation', 'music']",['JP'],NaN,tt16781874,7.5,19.0,1.853,7.0
4782,ts294227,Farzar,SHOW,Prince Fichael and his crew as they venture ou...,2022,TV-MA,27,"['scifi', 'animation', 'action', 'comedy']",['US'],1.0,tt14589904,5.3,929.0,47.136,6.5
4087,tm825952,Simon Amstell: Set Free,MOVIE,"Comedies Honest, introspective comic Simon Ams...",2019,NaN,51,['comedy'],['US'],NaN,tt10687642,6.5,815.0,1.823,6.1
5376,ts342479,Super PupZ,SHOW,Four pups with superpowers team up to help the...,2022,TV-Y7,29,"['comedy', 'family', 'fantasy', 'scifi']",['CA'],1.0,tt18469966,6.6,104.0,2.232,10.0
5663,tm983730,Amy Tan: Unintended Memoir,MOVIE,Amy Tan has established herself as one of Amer...,2021,NaN,101,['documentation'],['US'],NaN,tt10393074,7.7,321.0,4.121,8.3


观察到`genres`和`production_countries`两列不满足每个单元格只有一个值，  
但由于`production_countries`对分析目标没有影响，所以仅需要对`genres`进行拆分。  

In [29]:
cleaned_titles['genres'][1]

"['drama', 'crime']"

将`genres`中表示列表的字符串转换为实际的列表对象，再进行拆分。

In [31]:
cleaned_titles['genres'] = cleaned_titles['genres'].apply(lambda x :eval(x))

In [32]:
cleaned_titles = cleaned_titles.explode('genres')
cleaned_titles.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,documentation,['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,crime,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,drama,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,action,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300


- 对`credits.csv`演职员表进行评估，随机筛选10行观察

In [14]:
original_credits_data.sample(10)

,person_id,id,name,character,role
46260,1702305,tm983787,Uliana Tabaka,Club Goer (uncredited),ACTOR
52659,59976,tm890674,Barack Obama,Self,ACTOR
75508,2357965,tm856634,Robert McCrea,Shivados Gangster,ACTOR
15325,206327,tm71992,Babu Antony,Victor Rosetti,ACTOR
20944,931209,tm120609,Peter Banifaz,Sailor,ACTOR
77685,1307157,tm985747,Ying-Hung Lee,Bubble tea shop owner,ACTOR
31772,91185,ts41527,Poorna Jagannathan,Larin Inamdar,ACTOR
54861,231415,tm465408,Karen Kaia Livers,Helen Wayne,ACTOR
33685,59653,tm317110,Yoo Seung-mok,Mr. Kim,ACTOR
3448,92052,tm10739,Christopher Comrie,Beijing Riot Newscaster,ACTOR


`credits.csv`演职员表，符合整齐度，无需清洗。

### 干净度评估和清洗

- 对`titles.csv`电影表进行评估

In [33]:
cleaned_titles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15147 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    15147 non-null  object 
 1   title                 15146 non-null  object 
 2   type                  15147 non-null  object 
 3   description           15125 non-null  object 
 4   release_year          15147 non-null  int64  
 5   age_certification     9298 non-null   object 
 6   runtime               15147 non-null  int64  
 7   genres                15088 non-null  object 
 8   production_countries  15147 non-null  object 
 9   seasons               5923 non-null   float64
 10  imdb_id               14525 non-null  object 
 11  imdb_score            14399 non-null  float64
 12  imdb_votes            14375 non-null  float64
 13  tmdb_popularity       14995 non-null  float64
 14  tmdb_score            14614 non-null  float64
dtypes: float64(5), int64(2), 

#### 缺失数据

`title` `description` `age_certification` `genres` `seasons` `imdb_id` `imdb_score ` `imdb_votes ` `tmdb_popularity` `tmdb_score `    
经观产上述列存在缺失值，根据分析目标出发，仅有`genres``imdb_score `三列需要具体分析。  
- `imdb_score `缺失748行数据
- `genres`缺失59行数据

筛选`imdb_score`为空值的观察值观察

In [34]:
cleaned_titles[cleaned_titles['imdb_score'].isnull()]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,documentation,['US'],1.0,NaN,NaN,NaN,0.600,NaN
75,tm132164,Bill Hicks: Sane Man,MOVIE,Sane Man was filmed before Bill recorded ‘Dang...,1989,R,80,comedy,['US'],NaN,NaN,NaN,NaN,3.377,7.5
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991,TV-G,18,documentation,['JP'],12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991,TV-G,18,family,['JP'],12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991,TV-G,18,reality,['JP'],12.0,NaN,NaN,NaN,7.730,7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5810,tm1225897,Social Man,MOVIE,Two competitive social media Influencers go he...,2021,NaN,96,drama,[],NaN,tt20198164,NaN,NaN,NaN,NaN
5833,ts307884,HQ Barbers,SHOW,When a family run barber shop in the heart of ...,2021,TV-14,24,comedy,['NG'],1.0,NaN,NaN,NaN,0.840,NaN
5840,tm1216735,Sun of the Soil,MOVIE,"In 14th-century Mali, an ambitious young royal...",2022,NaN,26,NaN,[],NaN,NaN,NaN,NaN,1.179,7.0
5844,tm1074617,Bling Empire - The Afterparty,MOVIE,"The stars of ""Bling Empire"" discuss the show's...",2021,NaN,35,NaN,['US'],NaN,NaN,NaN,NaN,NaN,NaN


由于缺失分析所需的核心数据`imdb_score`，我们将把这些观察值删除，并查看删除后该列空缺值个数和：

In [41]:
cleaned_titles.dropna(subset = ['imdb_score'] , inplace = True )
cleaned_titles['imdb_score'].isnull().sum()

0

筛选genres为空值的观察值观察

In [43]:
cleaned_titles.query('genres.isnull()')

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1813,ts77824,My Next Guest Needs No Introduction With David...,SHOW,TV legend David Letterman teams up with fascin...,2018,TV-MA,50,NaN,['US'],4.0,tt7829834,7.8,5581.0,8.217,7.6
1939,ts215037,Minecraft: Story Mode,SHOW,"MInecraft: Story Mode is an interactive, anima...",2018,TV-PG,52,NaN,['US'],1.0,tt10498322,5.6,347.0,NaN,NaN
2386,ts74805,A Little Help with Carol Burnett,SHOW,In this unscripted series starring comedy lege...,2018,TV-G,24,NaN,['US'],1.0,tt7204366,6.3,237.0,1.621,6.2
2658,ts265844,#ABtalks,SHOW,#ABtalks is a YouTube interview show hosted by...,2018,TV-PG,68,NaN,[],1.0,tt12635254,9.6,7.0,NaN,NaN
4274,tm1172010,The Lockdown Plan,MOVIE,NaN,2020,NaN,49,NaN,[],NaN,tt13079112,6.5,NaN,NaN,NaN
4648,tm1113921,In Vitro,MOVIE,'In Vitro' is an otherworldly rumination on me...,2019,NaN,27,NaN,[],NaN,tt10545994,7.7,NaN,NaN,NaN


由于缺失分析所需的核心数据`genres`，我们将把这些观察值删除，并查看删除后该列空缺值个数和：

In [44]:
cleaned_titles.dropna(subset = ['genres'] , inplace = True )
cleaned_titles['genres'].isnull().sum()

0

#### 重复数据

In [48]:
cleaned_titles[cleaned_titles.duplicated()]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score


`titles.csv`影片表不存在重复数据

#### 不一致数据

根据分析目的出发，我们需要对影片流派`genres`进行聚合分析，所以需要对`genres`进行不一致数据评估

In [52]:
cleaned_titles['genres'].value_counts()

genres
drama            2827
comedy           2218
thriller         1180
action           1109
romance           955
crime             909
documentation     859
family            652
animation         630
fantasy           619
scifi             560
european          428
horror            366
history           254
music             244
reality           219
sport             170
war               155
western            39
Name: count, dtype: int64

从上面看出，`genres`列里并不存在不一致数据，各个值都在指代不同的流派。无需清洗

#### 无效或失效数据

首先转换`title.csv`表中`release_year`列的格式为日期。

In [53]:
cleaned_titles['release_year'] = pd.to_datetime(cleaned_titles["release_year"], format='%Y')
cleaned_titles['release_year']

1      1976-01-01
1      1976-01-01
2      1972-01-01
2      1972-01-01
2      1972-01-01
          ...    
5846   2021-01-01
5847   2021-01-01
5849   2021-01-01
5849   2021-01-01
5849   2021-01-01
Name: release_year, Length: 14393, dtype: datetime64[ns]

In [54]:
cleaned_titles.describe()

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
count,14393,14393.000000,5655.000000,14393.000000,1.437100e+04,14268.000000,13975.000000
mean,2015-12-30 06:28:41.336760832,78.326270,2.424934,6.534802,2.836162e+04,30.008057,6.885143
min,1954-01-01 00:00:00,0.000000,1.000000,1.500000,5.000000e+00,0.600000,1.000000
25%,2015-01-01 00:00:00,44.000000,1.000000,5.800000,7.080000e+02,3.911750,6.200000
50%,2018-01-01 00:00:00,87.000000,1.000000,6.600000,3.166000e+03,9.666000,6.985000
75%,2020-01-01 00:00:00,106.000000,3.000000,7.400000,1.432100e+04,22.766000,7.600000
max,2022-01-01 00:00:00,225.000000,42.000000,9.500000,2.294231e+06,2274.044000,10.000000
std,NaN,40.555015,2.867856,1.157513,1.041849e+05,99.359539,1.112448


从以上统计信息来看，`cleaned_titles`里不存在脱离现实意义的数值。

- 对`credits.csv`演职员表进行评估

In [55]:
cleaned_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77801 non-null  int64 
 1   id         77801 non-null  object
 2   name       77801 non-null  object
 3   character  68029 non-null  object
 4   role       77801 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


从输出结果来看，`cleaned_credits`数据共有77801条观察值，其中`character`变量存在缺失值，将在后续进行评估和清理。

此外，`person_id`表示演职员ID，数据类型不应为数字，应为字符串，所以需要进行数据格式转换。

In [56]:
cleaned_credits['person_id'] = cleaned_credits['person_id'].astype(str)
cleaned_credits['person_id']

0           3748
1          14658
2           7064
3           3739
4          48933
          ...   
77796     736339
77797     399499
77798     373198
77799     378132
77800    1950416
Name: person_id, Length: 77801, dtype: object

#### 缺失数据

In [57]:
cleaned_credits[cleaned_credits['character'].isnull()]

,person_id,id,name,character,role
36,3308,tm84618,Martin Scorsese,NaN,DIRECTOR
59,17727,tm154986,John Boorman,NaN,DIRECTOR
106,11475,tm127384,Terry Jones,NaN,DIRECTOR
107,11473,tm127384,Terry Gilliam,NaN,DIRECTOR
162,1063,tm120801,Robert Aldrich,NaN,DIRECTOR
...,...,...,...,...,...
77776,2363022,tm1097142,Mohamed El-Arkan,NaN,ACTOR
77777,1827884,tm1097142,Mohamed Bakir,NaN,DIRECTOR
77783,678884,tm1014599,Segun Arinze,NaN,ACTOR
77789,1962840,tm1014599,Seyi Babatope,NaN,DIRECTOR


`cleaned_credits`的缺失数据，其中只有`character`变量存在缺失值。

角色名并不影响我们挖掘各个流派中的高IMDB评分作品演员，并且此变量缺失也有可能因为演职员类别是导演，没有对应角色，因此可以保留`character`变量值存在空缺的观察值。

#### 重复数据

In [60]:
cleaned_credits[cleaned_credits.duplicated()]

,person_id,id,name,character,role


输出结果为0，说明不存在重复值。

#### 不一致数据

针对`original_credits`，不一致数据可能存在于`role`中，我们将查看是否存在多个不同值指代同一演职员类型的情况。

In [61]:
cleaned_credits['role'].value_counts()

role
ACTOR       73251
DIRECTOR     4550
Name: count, dtype: int64

从以上输出结果来看，`role`只有两种可能的值，`ACTOR`或`DIRECTOR`，不存在不一致数据。我们可以把这列的类型转换为`Category`，好处是比字符串类型更节约内存空间，也能表明说值的类型有限。

In [64]:
cleaned_credits['role'] = cleaned_credits['role'].astype('category')
cleaned_credits['role'].head()

0    ACTOR
1    ACTOR
2    ACTOR
3    ACTOR
4    ACTOR
Name: role, dtype: category
Categories (2, object): ['ACTOR', 'DIRECTOR']

#### 无效或失效数据

In [65]:
cleaned_credits.describe()

,person_id,id,name,character,role
count,77801,77801,77801,68029,77801
unique,54589,5489,54314,47274,2
top,48004,tm32982,Boman Irani,Self,ACTOR
freq,25,208,25,1950,73251


`original_credits`由于不包含表示数值含义的变量，因此无需用`describe`检查。

## 整理数据

为了能同时获得流派与演员数据，我们需要把`cleaned_titles`和`cleaned_credits`，通过`id`作为键进行连接，因为两个数据表中`id`都是影视作品ID。  
对于连接后的表来说，缺失演员或者缺失评分和电影名，都属于无效数据，所以我们应该进行内连接。

In [67]:
credits_with_titles = pd.merge(cleaned_credits , cleaned_titles , on = 'id')
credits_with_titles.head()

,person_id,id,name,character,role,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
1,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
3,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
4,7064,tm84618,Albert Brooks,Tom,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179


我们只需要`role`为`ACTOR`的观察值，所以筛选掉所有`DIRECTOR`的观察值

In [69]:
actor_with_titles = credits_with_titles.query('role == "ACTOR"')

为了挖掘出各个流派中的高IMDB评分作品演员，我们需要先根据流派和演员进行分组。

对演员进行分组的时候，选择的是用`person_id`而不是`name`变量，原因是名字容易出现错拼或者重名的情况，演职员ID会比演员姓名更加准确地反映是哪位演员。

In [70]:
groupby_genres_and_person_id = actor_with_titles.groupby(['genres','person_id'])

分组后，我们只需要对`imdb_score`的值进行聚合计算，因此只提取`imdb_score`变量，然后调用`mean`，来计算各个流派影视作品中，每位演员参演作品的平均IMDB评分。

In [71]:
imdb_score_groupby_genres_and_person_id = groupby_genres_and_person_id['imdb_score'].mean()
imdb_score_groupby_genres_and_person_id

genres   person_id
action   1000         6.866667
         100007       7.000000
         100013       6.400000
         100019       6.500000
         100020       6.500000
                        ...   
western  993735       6.500000
         998673       7.300000
         998674       7.300000
         998675       7.300000
         99940        4.000000
Name: imdb_score, Length: 168881, dtype: float64

我们可以调用`reset_index`，对层次化索引进行重置，得到更加规整的DataFrame。

In [72]:
imdb_score_groupby_genres_and_person_id_df = imdb_score_groupby_genres_and_person_id.reset_index()
imdb_score_groupby_genres_and_person_id_df

,genres,person_id,imdb_score
0,action,1000,6.866667
1,action,100007,7.000000
2,action,100013,6.400000
3,action,100019,6.500000
4,action,100020,6.500000
...,...,...,...
168876,western,993735,6.500000
168877,western,998673,7.300000
168878,western,998674,7.300000
168879,western,998675,7.300000


现在针对流派和演员分组的IMDB评分数据已经整理好，可以进入后续的分析步骤了。

但我们当前可以继续做一些数据整理，比如对上面的结果再次进行分组，找出各个流派里演员作品最高的平均评分是多少、最高评分对应的演员名字是什么。

要得到这一结果，我们需要再次用`genres`进行分组，然后提取出`imdb_score`变量，计算其最大值。

In [74]:
genres_max_scores = imdb_score_groupby_genres_and_person_id_df.groupby('genres')['imdb_score'].max()
genres_max_scores 

genres
action           9.3
animation        9.3
comedy           9.2
crime            9.5
documentation    9.1
drama            9.5
european         8.9
family           9.3
fantasy          9.3
history          9.1
horror           9.0
music            8.8
reality          8.9
romance          9.2
scifi            9.3
sport            9.1
thriller         9.5
war              8.8
western          8.9
Name: imdb_score, dtype: float64

在我们知道最高分后，可以把以上结果和之前得到的`imdb_score_groupby_genres_and_person_id_df`再次进行连接，得到最高分对应的各个演员ID是什么，也就是这个最高平均分是哪位演员拿到的。

In [75]:
genres_max_score_with_person_id = pd.merge(genres_max_scores , imdb_score_groupby_genres_and_person_id_df , on = ['genres','imdb_score'])
genres_max_score_with_person_id

,genres,imdb_score,person_id
0,action,9.3,12790
1,action,9.3,1303
2,action,9.3,21033
3,action,9.3,336830
4,action,9.3,86591
...,...,...,...
131,war,8.8,826547
132,western,8.9,22311
133,western,8.9,28166
134,western,8.9,28180


从以上结果可以看出，最高分对应的演员不一定只有一位，可能有多位演员的平均得分相同。

为了得到演员ID所对应的演员名字，我们可以和`cleaned_credits`这个DataFrame进行连接。这个DataFrame还有其它列，我们只需要得到`person_id`和`name`的对应，所以可以先提取出那两列，并把重复行删除。

In [78]:
actor_id_with_names = cleaned_credits[['person_id','name']].drop_duplicates()
actor_id_with_names

,person_id,name
0,3748,Robert De Niro
1,14658,Jodie Foster
2,7064,Albert Brooks
3,3739,Harvey Keitel
4,48933,Cybill Shepherd
...,...,...
77796,736339,Adelaida Buscato
77797,399499,Luz Stella Luengas
77798,373198,Inés Prieto
77799,378132,Isabel Gaona


下一步就可以把`actor_id_with_names`与前面得到的`genres_max_score_with_person_id`进行连接，增加`name`变量，从而展示平均评分最高的演员名字。

In [80]:
genres_max_score_with_actor_name = pd.merge(genres_max_score_with_person_id , actor_id_with_names , on = 'person_id' , how = 'left')
genres_max_score_with_actor_name

,genres,imdb_score,person_id,name
0,action,9.3,12790,Olivia Hack
1,action,9.3,1303,Jessie Flower
2,action,9.3,21033,Zach Tyler
3,action,9.3,336830,André Sogliuzzo
4,action,9.3,86591,Cricket Leigh
...,...,...,...,...
131,war,8.8,826547,Yuto Uemura
132,western,8.9,22311,Koichi Yamadera
133,western,8.9,28166,Megumi Hayashibara
134,western,8.9,28180,Unsho Ishizuka
